In [1]:
import cv2
import numpy as np
import os
from random import randint, choice

To be applied in patches of 512

In [8]:
def apply_occlusions(input_folder, output_folder, min_occlusions=20, max_occlusions=30, min_radius=25, max_radius=55, border_margin=55):
    """
    Apply fake occlusions to plant root mask images for data augmentation.

    This function automates the generation of occlusions in plant root mask images to create training data for a model
    aimed at "healing" or reconstructing occluded parts of plant root masks. The occlusions are circular and applied randomly
    across the images, with constraints to prevent occlusion on image borders, root tips, and maintaining some images without occlusions.

    Parameters:
    - input_folder: str, the folder containing the original mask images.
    - output_folder: str, the folder where the occluded images will be saved.
    - min_occlusions: int, the minimum number of occlusions to apply to an image.
    - max_occlusions: int, the maximum number of occlusions to apply to an image.
    - min_radius: int, the minimum radius of the circular occlusions.
    - max_radius: int, the maximum radius of the circular occlusions.
    - border_margin: int, the margin from the image border within which occlusions should not be applied.

    The function saves the occluded images in the output folder, preserving the file names.
    """

    # Create the output folder if it does not exist.
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # List all images in the input folder.
    images = [f for f in os.listdir(input_folder) if f.endswith('.png') or f.endswith('.jpg')]
    root_mask_counter = 0  # Counter to track images with root masks for selective occlusion application.

    for img_name in images:
        img_path = os.path.join(input_folder, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Skip occlusion for completely black images or every 20th image with a root mask.
        if np.count_nonzero(img) == 0 or root_mask_counter % 20 == 0:
            cv2.imwrite(os.path.join(output_folder, img_name), img)
            if np.count_nonzero(img) > 0:
                root_mask_counter += 1
            continue

        root_mask_counter += 1
        height, width = img.shape
        mask = np.zeros_like(img)  # Initialize a mask to apply occlusions.

        num_occlusions = randint(min_occlusions, max_occlusions)  # Determine the number of occlusions to apply.

        for _ in range(num_occlusions):
            attempt = 0  # Track the number of attempts to find a suitable white pixel for occlusion.
            while attempt < 100:  # Limit attempts to prevent infinite loops.
                center_x = randint(border_margin, width - border_margin - 1)
                center_y = randint(border_margin, height - border_margin - 1)
                if img[center_y, center_x] == 255:  # Check if the pixel is part of the root mask.
                    radius = randint(min_radius, max_radius)
                    cv2.circle(mask, (center_x, center_y), radius, 255, -1)  # Apply the occlusion.
                    break
                attempt += 1

        # Apply the occlusions mask to the original image.
        occluded_img = cv2.bitwise_and(img, 255 - mask)
        # Save the occluded image in the output folder.
        cv2.imwrite(os.path.join(output_folder, img_name), occluded_img)


In [9]:
apply_occlusions('./test', './test_occluded')

In [9]:
# apply_occlusions('data_patched_512_checked_2/val_images/val', 'data_patched_512_checked_2/val_masks/val')

In [10]:
# apply_occlusions('data_patched_512_checked_2/train_images/train', 'data_patched_512_checked_2/train_masks/train')